# Setup

In [ ]:
# Load packages
library(tidyverse)

In [ ]:
# Load observed and permuted data
observed <- read_delim(snakemake@input[["obs"]], delim = '\t')
permuted <- purrr::map_dfr(snakemake@input[["perm"]], read_delim, delim ='\t')

# Analyses

## Observed and permuted score correlations

In [ ]:
calc_cor <- function(df){
    val <- cor(df$mean, observed$mean, method = "pearson")
    return(data.frame(cor = val))
}
cor_plot <- permuted %>% 
    group_split(iter) %>% 
    purrr::map_dfr(., calc_cor) %>% 
    ggplot(aes(x = cor)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of iterations") + xlab("Permuted vs. observed XP-nSL Pearson's correlation coefficient") +
        theme_classic() +
        theme(axis.title = element_text(size = 17),
              axis.text = element_text(size = 15))

ggsave(filename = snakemake@output[["cor_plot"]], plot = cor_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

## Comparing observed vs. permuted selected windows

### Positive selection in urban habitats

In [ ]:
urban_sel_permuted <- permuted %>% filter(direction == "Urban sel")
urban_sel_observed <- observed %>% filter(direction == "Urban sel")

urb_mean_plot <- ggplot() +
    geom_density(data = urban_sel_permuted, aes(x = mean, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = urban_sel_observed, aes(x = mean, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Mean windowed XP-nSL score") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    theme_classic() +
    theme(axis.title = element_text(size = 20),
)
urb_mean_plot
ggsave(filename = snakemake@output[["urb_mean_plot"]], plot = urb_mean_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
urb_prop_plot <- ggplot() +
    geom_density(data = urban_sel_permuted, aes(x = prop_outlier, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = urban_sel_observed, aes(x = prop_outlier, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Proportion of XP-nSL scores in window > 2") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    theme_classic() +
    theme(axis.title = element_text(size = 20),
          axis.text = element_text(size = 18))
urb_prop_plot
ggsave(filename = snakemake@output[["urb_prop_plot"]], plot = urb_prop_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
urban_percentile <- urban_sel_observed %>% 
    rowwise() %>% 
    mutate(mean_perm_perc = sum(mean >= urban_sel_permuted$mean) / nrow(urban_sel_permuted),
           prop_perm_perc = sum(prop_outlier >= urban_sel_permuted$prop_outlier) / nrow(urban_sel_permuted)) %>% 
    arrange(desc(mean_perm_perc)) %>% 
    dplyr::select(Chr, winID, start, end, winCenter, mean_perm_perc, prop_perm_perc)

write_delim(urban_percentile, snakemake@output[["urb_perc"]], delim = "\t")

## Positive selection in rural habitat

In [ ]:
rural_sel_permuted <- permuted %>% filter(direction == "Rural sel")
rural_sel_observed <- observed %>% filter(direction == "Rural sel")

rur_mean_plot <- ggplot() +
    geom_density(data = rural_sel_permuted, aes(x = mean, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = rural_sel_observed, aes(x = mean, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Mean windowed XP-nSL score") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    theme_classic() +
    theme(axis.title = element_text(size = 20),
          axis.text = element_text(size = 18))
rur_mean_plot
ggsave(filename = snakemake@output[["rur_mean_plot"]], plot = rur_mean_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
rur_prop_plot <- ggplot() +
    geom_density(data = rural_sel_permuted, aes(x = prop_outlier, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = rural_sel_observed, aes(x = prop_outlier, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Mean windowed XP-nSL score") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    theme_classic() +
    theme(axis.title = element_text(size = 20),
          axis.text = element_text(size = 18))
rur_prop_plot
ggsave(filename = snakemake@output[["rur_prop_plot"]], plot = rur_prop_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
rural_percentile <- rural_sel_observed %>% 
    rowwise() %>% 
    mutate(mean_perm_perc = sum(mean <= rural_sel_permuted$mean) / nrow(rural_sel_permuted),
           prop_perm_perc = sum(prop_outlier >= rural_sel_permuted$prop_outlier) / nrow(rural_sel_permuted)) %>% 
    arrange(desc(mean_perm_perc)) %>% 
    dplyr::select(Chr, winID, start, end, winCenter, mean_perm_perc, prop_perm_perc)

write_delim(rural_percentile, snakemake@output[["rur_perc"]], delim = "\t")